# Import libraries

In [82]:
from text_matcher.matcher import Text, Matcher
import json
import pandas as pd
import random
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 6]
#pd.set_option('display.max_colwidth', None)

# Load in our Data Files

🛑 Input a link to the json file of articles to run the spell-check on


In [83]:
# Load in the JSON file with our JSTOR articles and data from TextMatcher
# (Note: must have the file 'default.json' in the same directory as this notebook)
articles = pd.read_json('../algorithm-testing/jstor-gender-trouble-all-articles.jsonl', lines=True)

# Importing the spell-check package

Languages tested:
* English - ‘en’
* Spanish - ‘es’
* French - ‘fr’
* German - ‘de’

Make sure to run `pip install pyspellchecker` in terminal before running the cell below

Recommended upper bound of anomaly beyond 75% percentile based on normal distribution:

    mean = 0.12764241550612537
    n = 30
    std = 0.02330421008
    [25 percentile, 75 percentile]: [0.1119237258,0.1433611052]

    Reccomended bound: 85.66


In [162]:
from spellchecker import SpellChecker

# ‼️ 🛑 Make sure to change the variable below to the desired accuracy bound 🛑  ‼️
bound = 85.66

#initialize dictionary pairing readability scores and article id's
articles_read_scores = {}

#print(len(articles.index))

# goes through each row (article) in the dataframe:
for index in range(0,1):
    article_index = index

    # defining variables
    article_id  = articles['id'].loc[article_index] 
    article_text = articles['fullText'].loc[article_index]
    article_title = articles['title'].loc[article_index]

    # Assign the full text of this article to a variable called `cleaned_article_text`, with text-matcher's Text function
    cleaned_article_text = Text(article_text, article_title)

    word_list = cleaned_article_text.getTokens()
    
    #word_list = ((" ").join(cleaned_article_text)).split(" ")

    #word_list = random.sample(word_list, int(len(word_list)/float(1)))

    #word_list = word_list[:int(len(word_list)*6/10)]

    # finding the document language
    languages = ['en','fr','es','de']
    abbrev_word_list = word_list[:50]
    incorrect = []
    for lg in languages:
        spell = SpellChecker(language = lg)
        misspelled = spell.unknown(abbrev_word_list)
        incorrect.append(len(misspelled))
    lang = languages[incorrect.index(min(incorrect))]

    # find those words that may be misspelled
    spell = SpellChecker(language = lang)
    misspelled = spell.unknown(word_list)

    #output the readability score
    incorrect_percentage = float(len(misspelled))/len(word_list)

    if (1 - incorrect_percentage) < float(bound)/100:
        articles_read_scores[article_id] = [article_index, article_title, incorrect_percentage]

print(articles_read_scores)

{}


In [27]:
articles.head(3)

,abstract,creator,datePublished,docSubType,docType,fullText,id,identifier,isPartOf,issueNumber,...,provider,publicationYear,publisher,sourceCategory,tdmCategory,title,url,volumeNumber,wordCount,subTitle
0,This paper investigates the logic of explanati...,[Merje Kuus],2007-01-01,research-article,article,[Ubiquitous identities and elusive subjects: p...,http://www.jstor.org/stable/4640001,"[{'name': 'issn', 'value': '00202754'}, {'name...",Transactions of the Institute of British Geogr...,1,...,jstor,2007,Royal Geographical Society (with the Institute...,"[Geography, Social Sciences]",[Philosophy - Applied philosophy],Ubiquitous Identities and Elusive Subjects: Pu...,http://www.jstor.org/stable/4640001,32,9633,NaN
1,Seeking to elucidate understandings of sexual ...,[Rebecca M. Herzig],2000-10-01,research-article,article,[The Woman Beneath the Hair: Treating Hypertri...,http://www.jstor.org/stable/4316762,"[{'name': 'issn', 'value': '10400656'}, {'name...",NWSA Journal,3,...,jstor,2000,The Johns Hopkins University Press,"[Feminist & Women's Studies, Social Sciences]","[Social sciences - Communications, Philosophy ...",The Woman beneath the Hair: Treating Hypertric...,http://www.jstor.org/stable/4316762,12,7509,NaN
2,NaN,NaN,2003-05-01,research-article,article,[PÉRIPHÉRIES I 113 Références bibliographiques...,http://www.jstor.org/stable/40978757,"[{'name': 'issn', 'value': '11440821'}, {'name...",Rue Descartes,40,...,jstor,2003,Presses Universitaires de France,"[Humanities, Philosophy]",NaN,Références bibliographiques des ouvrages et ar...,http://www.jstor.org/stable/40978757,NaN,1510,NaN
